## Reasoning and Acting (ReAct)​

Agent to implement ReAct logic

In [2]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import AzureChatOpenAI

In [4]:
%pip install pypdf
%pip install pycryptodome
%pip install pymupdf


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/Underwriting Guidelines 319E 1.pdf")

data = loader.load()

data[0]

Document(metadata={'source': 'data/Underwriting Guidelines 319E 1.pdf', 'file_path': 'data/Underwriting Guidelines 319E 1.pdf', 'page': 0, 'total_pages': 27, 'format': 'PDF 1.7', 'title': 'Underwriting Guidelines', 'author': 'BMO Insurance', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 19.1 (Macintosh)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20240126101951-05'00'", 'modDate': "D:20240126111742-05'00'", 'trapped': '', 'encryption': 'Standard V5 R6 256-bit AES'}, page_content='BMO Insurance\nEffective Date:\nFebruary 26, 2024\n319E (2024/02/26)\nUnderwriting \nGuidelines\n')

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002", openai_api_version="2023-05-15")
vector = FAISS.from_documents(data, embeddings)
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "bmo_search",
    "Search for information about insurance underwriting. For any questions about insurance, you must use this tool!",
)

In [7]:
tools = [retriever_tool, TavilySearchResults(max_results=1)]

In [8]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt.pretty_print()

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [9]:
# Choose the LLM to use
llm = AzureChatOpenAI(
    openai_api_version="2024-06-01",
    azure_deployment="gpt-4o",
)

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [10]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
I need to determine what LangChain is. Since this question is not related to insurance, I should use the tavily_search_results_json tool to find comprehensive and accurate information.

Action: tavily_search_results_json
Action Input: "what is LangChain?"[{'url': 'https://python.langchain.com/docs/introduction/', 'content': "How to use tools in a chain How to migrate from legacy LangChain agents to LangGraph How to use chat models to call tools LangChain is a framework for developing applications powered by large language models (LLMs). Development: Build your applications using LangChain's open-source building blocks, components, and third-party integrations. langchain: Chains, agents, and retrieval strategies that make up an application's cognitive architecture. LangServe: Deploy LangChain chains as REST APIs. LangSmith: A developer platform that lets you debug, test, evaluate, and monitor LLM applications. Build stateful, multi-actor applicati

{'input': 'what is LangChain?',
 'output': 'LangChain is a framework for building applications powered by large language models, offering tools and integrations for developing, deploying, and monitoring these applications.'}

In [12]:
agent_executor.invoke(
    {"input": "What are the requirements for an Male applicant aged  70 for $9M ?"}
)



> Entering new AgentExecutor chain...
I need to gather specific information about the underwriting requirements for a male applicant aged 70 seeking $9 million in insurance coverage. To do this, I will use the tool specifically designed for obtaining information about insurance underwriting.

Action: bmo_search
Action Input: "requirements for a male applicant aged 70 for $9M insurance coverage"Reinstatements cont’d...
Life Insurance (Term) after 90 days from 
the lapse effective date for Non-Smoker 
– preferred or preferred rate classification
For amounts $100,000 to $500,000 and up to Age 65:
•	Long Form Health Certificate 167E
•	Urinalysis
For amounts greater than $500,000 or Age 65 and over:
•	Long Form Health Certificate 167E
•	Age and Amount Requirements
For all ages and amounts:
•	Long Form Health Certificate 167E
•	Age and Amount Requirements
Critical Illness Insurance within 90 days from 
the policy lapse effective date
For all ages and amounts:
•	Long Form Health Certificate

{'input': 'What are the requirements for an Male applicant aged  70 for $9M ?',
 'output': 'The underwriting requirements for a male applicant aged 70 seeking $9 million in insurance coverage include:\n- Tele-Interview & Vitals (TV3)\n- Blood Profile, Urinalysis, APS (32)\n- Financial: Third-party verification (F)\n- Motor Vehicle Report (M2)'}